# Import all your libraries

In [1]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
PINECONE_INDEX = os.getenv("PINECONE_INDEX")

openai.api_key = OPENAI_API_KEY
openai.Engine.list()  # check we have authenticated

<OpenAIObject list at 0x11ed5fb30> JSON: {
  "data": [
    {
      "created": null,
      "id": "babbage",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "davinci",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-davinci-edit-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "babbage-code-search-code",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-similarity-babbage-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "code-davinci-edit-001",
      "object": "engine",
      "ow

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

In [3]:
import os

faq = None
for path in os.listdir():
    if path == "UTDFAQ.csv":
        faq = path
    else:
        print("UTDFAQ.csv not found in current directory")
        faq = "FAQ Dataset.csv"
print(faq)

UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current directory
UTDFAQ.csv not found in current di

# Load the Dataset

In [4]:
from datasets import load_dataset
from rich import print
from rich import print

faq_dataset = load_dataset(
    "csv", data_files=faq)
print(faq_dataset)
## remove all the None values
faq_dataset = faq_dataset.filter(lambda x: x['Question'] is not None and x['Answering'] is not None)
## reaplace all the word ARC to AccessAbility Resource Center and
'''
Office location: Administration Building, Room 2.224
Email: studentaccess@utdallas.edu (Do not email attachments, upload documents to utd.link/arcupload only.)
Phone: (972) 883-2098
Fax: Please don’t fax, use utd.link/arcupload
Mail: AD 30, 800 West Campbell Rd., Richardson TX 75080
'''
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('ARC', 'AccessAbility Resource Center'), 'Answering': x['Answering'].replace('ARC', 'AccessAbility Resource Center')})

Using custom data configuration default-b4021009cea19f83


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/ari/.cache/huggingface/datasets/csv/default-b4021009cea19f83/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answering', 'source', 'Label'],
        num_rows: 156
    })
})

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/156 [00:00<?, ?ex/s]

# Preprocess the dataset

## remove all the None values

In [5]:
## remove all the None values
faq_dataset = faq_dataset.filter(lambda x: x['Question'] is not None and x['Answering'] is not None)

  0%|          | 0/1 [00:00<?, ?ba/s]

## Replace all the ARC to Accessible Resource Center

In [6]:
# replace all the word UTD to University of Texas at Dallas
# replace all the ARC to AccessAbility Resource Center
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('UTD', 'University of Texas at Dallas'), 'Answering': x['Answering'].replace('UTD', 'University of Texas at Dallas')})
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('ARC', 'AccessAbility Resource Center'), 'Answering': x['Answering'].replace('ARC', 'AccessAbility Resource Center')})

  0%|          | 0/156 [00:00<?, ?ex/s]

  0%|          | 0/156 [00:00<?, ?ex/s]

# Load the Open AI Emebddigns

In [7]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables from .env.

OPENKEY_API = os.getenv("OPENAI_KEY")

#ORGANIZATION_ID = os.getenv("ORGANIZATION_ID")
#openai.organization = ORGANIZATION_ID
# get this from top-right dropdown on OpenAI under organization > settings
openai.api_key = OPENKEY_API
# get API key from top-right dropdown on OpenAI website
os.environ["OPENAI_API_KEY"] = OPENKEY_API
openai.Engine.list()  # check we have authenticated
print(openai.Engine.list())
## model of choices
MODEL = "text-embedding-ada-002"

<OpenAIObject list at 0x1463737c0> JSON: {
  "data": [
    {
      "created": null,
      "id": "babbage",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "davinci",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-davinci-edit-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "babbage-code-search-code",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-similarity-babbage-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "code-davinci-edit-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-davinci-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-davinci-003",
      "object": "engine",
      "owner": "openai-internal",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "ada",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "babbage-code-search-text",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "babbage-similarity",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "code-search-babbage-text-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-curie-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "whisper-1",
      "object": "engine",
      "owner": "openai-internal",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "code-search-babbage-code-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-ada-001",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-embedding-ada-002",
      "object": "engine",
      "owner": "openai-internal",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "text-similarity-ada-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "curie-instruct-beta",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "ada-code-search-code",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "ada-similarity",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "gpt-3.5-turbo",
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true
    },
    {
      "created": null,
      "id": "code-search-ada-text-001",
      "object": "engine",
      "owner": "openai-dev",
      "permissions": null,
      "ready": true
    },
    {
      "creat

# Ini the Pinecone Client

In [8]:
import pinecone
PINECONE_APIKEY = os.getenv("PINECONE_APIKEY")
if PINECONE_APIKEY is None:
    raise Exception("PINECONE_API_KEY not found in environment variables add the Pinecone API key to the .env file")
# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key = PINECONE_APIKEY,
    environment="us-east-1-aws"
)

In [9]:
def set_env():
    #setting up OpenAI Api
    OPENKEY_API = os.getenv("OPENAI_KEY")
    ORGANIZATION_ID = os.getenv("ORGANIZATION_ID")
    os.environ["OPENAI_API_KEY"] = OPENKEY_API
    openai.organization = ORGANIZATION_ID
    # get this from top-right dropdown on OpenAI under organization > settings
    openai.api_key = OPENKEY_API
    # get API key from top-right dropdown on OpenAI website

    openai.Engine.list()  # check we have authenticated

    #setting pup pinecon
    PINECONE_APIKEY = os.getenv("PINECONE_APIKEY")
    # initialize connection to pinecone (get API key at app.pinecone.io)
    pinecone.init(
        api_key = PINECONE_APIKEY ,
        environment="us-east1-gcp" , 
    )
    assert len(pinecone.list_indexes()) > 0, "No indexes found in your Pinecone account. Please create an index and try again."

    print(pinecone.list_indexes())
    if pinecone.list_indexes()[0] != "utd-chatbot":
        print("Please create an index named 'utd-chatbot")


    print(f'The list of pinecone index {pinecone.list_indexes()}')
    if pinecone.list_indexes()[0] != "utd-chatbot":
        print("Please create an index named 'utd-chatbot")

    ## I the name of the vector database
    return  pinecone.Index(index_name = pinecone.list_indexes()[0]) , pinecone.list_indexes()[0] , pinecone.describe_index(pinecone.list_indexes()[0])

pinecone_index , index_name , index_description = set_env()

['faq-dataset']

Please create an index named 'utd-chatbot

The list of pinecone index ['faq-dataset']

Please create an index named 'utd-chatbot

In [10]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 176}},
 'total_vector_count': 176}

# Role Prompt

In [11]:
role_prompt = """
Please act as a University of Texas at Dallas Counselor. I will provide you with an individual 
looking for guidance at the University of Texas at Dallas, and your task is to help them 
solve their problem\n
"""

In [12]:
def post_process(ans):
    if "ARC" in ans:
        ans = ans.replace("ARC", "AccessAbility Resource Center")
        ans += """ 
        Office location: Administration Building, Room 2.224 \n
        Email: studentaccess@utdallas.edu (Do not email attachments, upload documents to utd.link/arcupload only.) \n
        Phone: (972) 883-2098 \n
        Fax: Please don’t fax, use utd.link/arcupload \n
        Mail: AD 30, 800 West Campbell Rd., Richardson TX 75080    Office location: Administration Building, Room 2.224\n
        """
    return ans

In [13]:

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pinecone import Pinecone

def query_vector_database(query):
    embeddings = OpenAIEmbeddings()

    # vectorstore = Pinecone.from_documents(documents, embeddings, index_name=INDEX_NAME) # only used for initial index creation, upserts document embeddings as well as the documents themselves
    vectorstore = Pinecone( pinecone_index, embeddings.embed_query, "text") # use this for subsequent runs

    docs = vectorstore.similarity_search_with_score(query)

    res = []
    for doc in docs:
      answer = faq_dataset['train'].filter(lambda x: x['Question'] == doc[0].page_content) # get row with the corresponding question in query
      print(answer)
      res.append({"Question": f"{answer['Question'][0]}", "Answer": f"{answer['Answering'][0]}", "URL": f"{answer['URL'][0]}"}) # adds a dictionary of the row to list
    return res

In [14]:
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone( pinecone_index, embeddings.embed_query, "text") # use this for subsequent runs
docs = vectorstore.similarity_search_with_score("I have ADHD and I need help with my classes.")
print(docs)

[
    (
        Document(
            page_content='My professor says they cannot find the space or time for me to take my test with 
accommodations. Guess I will just take it with the class and hope for the best?',
            metadata={
                'url': 'https://accessability.utdallas.edu/student-accommodations/frequently-asked-questions/'
            }
        ),
        0.824971378
    ),
    (
        Document(
            page_content='My professor says they cannot find the space or time for me to take my test with 
accommodations. Guess I will just take it with the class and hope for the best?',
            metadata={}
        ),
        0.824911535
    ),
    (
        Document(
            page_content='My class has attendance requirements because students are graded while I observe what 
they are doing and they must access specialized lab equipment that cannot be moved. How can I allow remote 
learning? ',
            metadata={}
        ),
        0.814337552
    ),
    (Document(page_content='I would like to change my major. What do I need to do?', metadata={}), 0.806916952)
]

In [15]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

def create_an_standard_qa_prompt(res):
    example_prompt = PromptTemplate(
        input_variables=["Question", "Answer", "URL"], 
        template="Question: {Question}\n{Answer}\nSource:{URL}"
    )

    fewShotPrompt = FewShotPromptTemplate(
        examples=res,
        example_prompt=example_prompt,
        suffix="Question: {input}",
        input_variables=["input"]
    )

    system_message_prompt = SystemMessage(content=role_prompt)
    human_message_prompt = HumanMessagePromptTemplate(prompt=fewShotPrompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    type(chat_prompt)
    return chat_prompt

In [16]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

In [17]:
from langchain import LLMChain

def called_llm_decoder_model(query, prompt):
  chain = LLMChain(llm=chat, prompt=prompt)
  output = chain.run(input=query)
  return output


In [18]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT,
)

In [19]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

index = pinecone.Index(PINECONE_INDEX)
vectorstore = Pinecone(index, OpenAIEmbeddings().embed_query, text_key="text")

In [20]:
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

conv_chain = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0), 
    vectorstore.as_retriever(k = 3),
    return_source_documents=True,
    max_tokens_limit = 4096-300
)

# Gradio

In [21]:
import gradio as gr
chat_history = []

def utd_chatbot( question ):
    result = conv_chain({"question": question, "chat_history": chat_history})
    return result['answer']

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def respond(question, chat_history):
        bot_message = conv_chain({"question": question, "chat_history": chat_history}) 
        bot_message = bot_message['answer']
        chat_history.append((question, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

In [22]:
## Lauch of Gradio
#demo = gr.Interface(fn=utd_chatbot, inputs="text", outputs="text")
demo.launch( share = True )

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d64983f4-1b5c-40ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [23]:
# Close the a demo
demo.close()

Closing server running on port: 7860


# Use Language Chain to Generate one Example

In [24]:
from langchain import PromptTemplate


template = """
I want you to act as a naming consultant for new companies.

Here are some examples of good company names:

- search engine, Google
- social media, Facebook
- video sharing, YouTube

The name should be short, catchy and easy to remember.

What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2)
from rich import print
print( llm("Tell me about yourself") ) 

My name is John and I'm currently a student studying computer science. I'm passionate about learning new 
technologies and applying them to solve real-life problems. I'm also an avid fan of sports, particularly 
basketball. In my free time, I like to explore different cultures and cuisines, and I'm always looking for new 
experiences.